## Ten seconds is too long for an inner join between regions & hits tables - python version

- here we discover that doing the join (merge) in python takes ??? seconds

In [1]:
import pandas as pd, psycopg2 as psql, time

In [2]:
dbpiq = psql.connect("dbname=piqTest user=pshannon")

In [3]:
query = "select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';"
pd.read_sql_query(query, dbpiq).loc[:, "relname"].tolist()

['regions', 'hits']

### what do the tables look like?

In [4]:
tbl_counts = pd.read_sql_query("select count(*) from regions", dbpiq).append(
             pd.read_sql_query("select count(*) from hits", dbpiq))
tbl_counts.index = ["regions", "hits"]
tbl_counts

,count
regions,1158121
hits,30072096


In [5]:
pd.read_sql_query("select * from regions limit 3", dbpiq)

,loc,chrom,start,stop
0,chr21:5011471-5011482,chr21,5011471,5011482
1,chr21:5018173-5018184,chr21,5018173,5018184
2,chr21:5057475-5057486,chr21,5057475,5057486


In [6]:
pd.read_sql_query("select * from hits limit 3", dbpiq)

,loc,type,name,strand,sample_id,method,provenance,score1,score2,score3,score4,score5,score6
0,chr21:9650860-9650871,motif.in.footprint,MA0032.2,+,ENCSR000EJJ,piq,piq.minid.tbd,10.1972,-0.905663,3.06436,0.548674,None,None
1,chr21:9651991-9652002,motif.in.footprint,MA0032.2,+,ENCSR000EJJ,piq,piq.minid.tbd,10.2452,-0.987957,2.46424,0.539790,None,None
2,chr21:9653310-9653321,motif.in.footprint,MA0032.2,-,ENCSR000EJJ,piq,piq.minid.tbd,10.8088,-0.999619,-18.29330,0.549584,None,None


In [7]:
start_time = time.time()
query = "select count(*) from regions where chrom='chr21' and start > 5010000 and stop < 5010500"
tbl = pd.read_sql_query(query, dbpiq)
print("--- %5.2f seconds ---" % (time.time() - start_time))

---  0.20 seconds ---


In [8]:
def getHits(db, chrom, start, stop):
  query = """select * from regions r
    inner join hits h on r.loc = h.loc
    where r.chrom = '%s' and r.start > %d and r.stop < %d""" % (chrom, start, stop)
  tbl = pd.read_sql_query(query, db)
  # tbl.drop(tbl.columns[0], 1, inplace=True)  # duplicate "loc" columns.  drop the first
  return(tbl) 

### before indexing the hits table:
<pre>
 user  system elapsed 
0.005   0.000  11.009 
 dim: 36 16   
<pre>

In [9]:
start_time = time.time()
tbl = getHits(dbpiq, "chr21", 15010000, 15010030)
print("--- %5.2f seconds ---" % (time.time() - start_time))
tbl.shape

---  8.77 seconds ---


(18, 17)

### ~ 10secs on the join seems too long
Break the join up into two queries, time them each

In [10]:
start_time = time.time()
tbl = pd.read_sql_query("select loc from regions where chrom='chr21' and start > 15010014 and stop < 150100100", dbpiq)
print("--- %5.2f seconds ---" % (time.time() - start_time))

---  0.77 seconds ---


In [11]:
start_time = time.time()
tbl = pd.read_sql_query("select * from hits where loc='chr21:15010014-15010020'", dbpiq)
print("--- %5.2f seconds ---" % (time.time() - start_time))

---  0.03 seconds ---


In [12]:
tbl.shape

(18, 13)

### Do the 'join' (aka 'merge') in pythoh:  50x faster

In [13]:
def getHits2(db, chrom, start, stop):

   queryRegions_0 = "select loc, chrom, start, stop from regions"
   queryRegions_1 = "where chrom='%s' and start >= %d and stop <= %d" % (chrom, start, stop)
   queryRegions = "%s %s" % (queryRegions_0, queryRegions_1)
   tbl_regions = pd.read_sql_query(queryRegions, db)
   locs = tbl_regions.loc[:, "loc"].tolist()

   queryHits = "select * from hits where loc in %s" % str(locs)
   queryHits = queryHits.replace("[", "(").replace("]", ")")
   tbl_hits = pd.read_sql_query(queryHits, db)
   return(pd.merge(tbl_regions, tbl_hits, on="loc"))


In [16]:
start_time = time.time()
tbl = getHits2(dbpiq, "chr21", 15010000, 15010030)
print("--- %5.2f seconds ---" % (time.time() - start_time))
tbl.shape

---  0.19 seconds ---


(18, 16)

In [17]:
tbl.loc[0:4, "chrom":"score1"]

,chrom,start,stop,type,name,strand,sample_id,method,provenance,score1
0,chr21,15010014,15010020,motif.in.footprint,MA0442.1,-,ENCSR000DBY,piq,piq.minid.tbd,6.93175
1,chr21,15010014,15010020,motif.in.footprint,MA0442.1,-,ENCSR000DBZ,piq,piq.minid.tbd,6.93175
2,chr21,15010014,15010020,motif.in.footprint,MA0442.1,-,ENCSR000DCA,piq,piq.minid.tbd,6.93175
3,chr21,15010014,15010020,motif.in.footprint,MA0442.1,-,ENCSR000DCB,piq,piq.minid.tbd,6.93175
4,chr21,15010014,15010020,motif.in.footprint,MA0442.1,-,ENCSR000DCC,piq,piq.minid.tbd,6.93175
